In [29]:
#
#                                               Logan Caldwell, 2019
#

# Importing dependencies
from splinter import Browser
import requests

In [2]:
from bs4 import BeautifulSoup as bs

In [ ]:
import datetime as dt

In [3]:
# Getting chromedriver path
!which chromedriver

/c/Users/lscca/bin/chromedriver


In [4]:
# Giving executable path and init chrome browser in splinter
executable_path = {'executable_path': 'C:/Users/lscca/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

In [5]:
#
#
#
#                                               To mars news site
#
#
#


url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Safe delay while loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [6]:
# Convert the html to soup object


html = browser.html
news_soup = bs(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [7]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8438/nasas-mro-completes-60000-trips-around-mars/" target="_self">NASA's MRO Completes 60,000 Trips Around Mars</a></div>

In [8]:
# Using parent element to find the first 'a' tag and saving it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()



news_title

"NASA's MRO Completes 60,000 Trips Around Mars"

In [9]:
# Using parent element to find paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'The orbiting spacecraft is also about to set a record for data relayed from the Martian surface.'

In [10]:
#
#
#
#                                  Getting featured images - Twitter site
#
#
#

In [11]:
# Linking url to splinter via browser/chromedriver

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [12]:
# Find full image button, then click it


full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [13]:
# Find more info button and click it


browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

In [14]:
# Parsing the resulting html with soup (bs)


html = browser.html
img_soup = bs(html, 'html.parser')

In [15]:
# find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA18087_hires.jpg'

In [16]:
# Using the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18087_hires.jpg'

In [17]:
#
#
#
#                                               Mars weather
#
#
#

In [18]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [19]:
# Retrieving html code and parsing via BeautifulSoup (bs)

html = browser.html
weather_soup = bs(html, 'html.parser')

In [20]:
# Finding a tweet with the "data-name" ≡ "Mars Weather"



mars_weather_tweet = weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})



In [21]:
# Searching within the tweet for the p tag containing the tweet text

mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
mars_weather

'InSight sol 165 (2019-05-15) low -100.3ºC (-148.6ºF) high -18.2ºC (-0.7ºF)\nwinds from the SW at 4.6 m/s (10.4 mph) gusting to 13.7 m/s (30.6 mph)\npressure at 7.50 hPapic.twitter.com/7NMgdAkFA8'

In [ ]:
#
#
#
#                                       Getting mars hemisphere images
#
#
#

In [22]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [23]:
hemisphere_image_urls = []

# Getting a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")

# Looping through those links, clicking the link, finding the sample anchor, returning the href
for i in range(len(links)):
    hemisphere = {}
    
    # Finding the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    
    # Finding the Sample image anchor tag and extracting the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Getting Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Appending hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Navigating backwards in browser
    browser.back()
    

In [24]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [25]:
#
#
#
#                                               Getting mars facts
#
#
#

In [26]:
import pandas as pd
df = pd.read_html('https://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [27]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

In [28]:
browser.quit()